## Import Tools

In [2]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import seaborn as sn
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

## Dataset Processing 

### Read in the original dataset 

In [3]:
train_dl_origin = torch.load('Dataset/train_dl.pt')
valid_dl_origin = torch.load('Dataset/valid_dl.pt')

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

In [4]:
print(train_label)

tensor([1, 1, 0,  ..., 0, 0, 1])


In [5]:
print(train_label.shape)

torch.Size([15000])


### CSI Processing: Take Modulus of complex matrices

In [6]:
train_CSI_modulus = torch.abs(train_CSI)
valid_CSI_modulus = torch.abs(valid_CSI)

In [62]:
print(train_CSI_modulus)

tensor([[[[100.6578, 124.7878, 106.1179,  ..., 304.7704, 299.6064, 324.3594],
          [132.8157, 106.6771,  91.2688,  ..., 269.1561, 323.5568, 299.9617],
          [129.1395, 148.4756, 170.0735,  ..., 399.8112, 407.4420, 402.0112],
          [ 74.0000,  71.4493,  59.3633,  ..., 134.0149, 129.6919, 124.0363]]],


        [[[177.0198, 170.4963, 169.1065,  ...,  46.6154,  37.6431,  64.4981],
          [143.6802, 143.0874,  88.0909,  ...,  44.0454,  22.2036,  27.6586],
          [ 97.8008,  80.7527,  71.7008,  ...,  32.2025,  22.4722,  39.3573],
          [ 39.8121,  45.7930,  31.6228,  ...,  16.5529,   8.0623,  25.6125]]],


        [[[411.3940, 421.5412, 380.1276,  ..., 509.8431, 550.0582, 539.8120],
          [366.8079, 387.3629, 353.0340,  ..., 596.1241, 619.6975, 605.5353],
          [574.8991, 593.8560, 612.0008,  ..., 928.9521, 923.3618, 914.0552],
          [289.8362, 287.2368, 281.0427,  ..., 354.9113, 339.0634, 333.9461]]],


        ...,


        [[[296.5889, 288.2672, 292.76

In [7]:
print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


###  CSI Processing: Normalize to [0,1]

In [8]:
# Min-Max Scaling
min_value = torch.min(train_CSI_modulus)
max_value = torch.max(train_CSI_modulus)

normalized_train_CSI_modulus = (train_CSI_modulus - min_value) / (max_value - min_value)
normalized_valid_CSI_modulus = (valid_CSI_modulus - min_value) / (max_value - min_value)


### ML Classifcation w/ KNN

In [9]:
# Convert 4d to 2d
train_data_2d = normalized_train_CSI_modulus.view(normalized_train_CSI_modulus.size(0), -1)  # Reshape to (15000, 4 * 1632)
valid_data_2d = normalized_valid_CSI_modulus.view(normalized_valid_CSI_modulus.size(0), -1)  # Reshape to (15000, 4 * 1632)

#### Below tests multiple k-values
    Don't recommend running as it takes a long time

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
sq = int(math.sqrt(15000))
accs = {}
for i in range(sq-30, sq+1, 10): 
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(train_data_2d, train_label)
    print('Accuracy of KNN classifier on training set: {:.2f}'.format(knn.score(train_data_2d, train_label)))
    accs[i] = knn.score(train_data_2d, train_label)
print("Best K-Neighbors: ", max(accs, key=accs.get))


In [15]:
from sklearn.neighbors import KNeighborsClassifier
sq = int(math.sqrt(15000))
knn = KNeighborsClassifier(n_neighbors = sq - 35)
#Train the model using the training sets
knn.fit(train_data_2d, train_label)
#Test the model using the testing sets
print('Accuracy of KNN classifier on training set: {:.2f}'.format(knn.score(train_data_2d, train_label)))
print('Accuracy of KNN classifier on valid set: {:.2f}'.format(knn.score(valid_data_2d, valid_label)))

C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:200: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)

Accuracy of KNN classifier on training set: 0.85
Accuracy of KNN classifier on valid set: 0.84


C:\Users\Joey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


### ANN approach (MLP)

- Instantiate a Neural Network Model

In [19]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4*1632, 300)  # input layer (4*1632 nodes) -> hidden layer (100 nodes)
        self.layer2 = nn.Linear(300, 300)  # hidden layer (100 nodes) -> hidden layer (100 nodes)
        self.layer3 = nn.Linear(300, 100)  # hidden layer (100 nodes) -> hidden layer (100 nodes)
        self.layer4 = nn.Linear(100, 1)  # hidden layer (100 nodes) -> output layer (1 nodes)

    def forward(self, x):
        return self.layer4(F.relu(self.layer3(F.relu(self.layer2(F.relu(self.layer1(x)))))))

net = Net()
print(net)

Net(
  (layer1): Linear(in_features=6528, out_features=300, bias=True)
  (layer2): Linear(in_features=300, out_features=300, bias=True)
  (layer3): Linear(in_features=300, out_features=100, bias=True)
  (layer4): Linear(in_features=100, out_features=1, bias=True)
)


- Add a loss function and an optimizer

In [20]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()  # define the loss function
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)  # define the optimizer

- Train the neural network

In [21]:
train_data_2d = torch.tensor(train_data_2d, dtype=torch.float32)
train_label = torch.tensor(train_label, dtype=torch.float32)
train_label = train_label.view(-1, 1)

num_epochs = 300
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = net(train_data_2d)
    loss = criterion(output, train_label)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

C:\Users\Joey\AppData\Local\Temp\ipykernel_2972\4240162415.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data_2d = torch.tensor(train_data_2d, dtype=torch.float32)
C:\Users\Joey\AppData\Local\Temp\ipykernel_2972\4240162415.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_label = torch.tensor(train_label, dtype=torch.float32)


Epoch [1/300], Loss: 0.6729807257652283
Epoch [2/300], Loss: 0.6725283861160278
Epoch [3/300], Loss: 0.67167067527771
Epoch [4/300], Loss: 0.6704527139663696
Epoch [5/300], Loss: 0.6689159274101257
Epoch [6/300], Loss: 0.6671010255813599
Epoch [7/300], Loss: 0.6650528907775879
Epoch [8/300], Loss: 0.6628193855285645
Epoch [9/300], Loss: 0.6604406833648682
Epoch [10/300], Loss: 0.6579378247261047
Epoch [11/300], Loss: 0.6553328633308411
Epoch [12/300], Loss: 0.6526481509208679
Epoch [13/300], Loss: 0.6498970985412598
Epoch [14/300], Loss: 0.6470907330513
Epoch [15/300], Loss: 0.644235372543335
Epoch [16/300], Loss: 0.6413372159004211
Epoch [17/300], Loss: 0.6384027600288391
Epoch [18/300], Loss: 0.6354385614395142
Epoch [19/300], Loss: 0.6324502825737
Epoch [20/300], Loss: 0.6294431686401367
Epoch [21/300], Loss: 0.6264225244522095
Epoch [22/300], Loss: 0.6233915090560913
Epoch [23/300], Loss: 0.6203547120094299
Epoch [24/300], Loss: 0.6173157691955566
Epoch [25/300], Loss: 0.6142776012

- Test the Neural Network

In [23]:
def test_accuracy(model, test_data, test_labels):
    model.eval()

    with torch.no_grad():
        test_predictions = model(test_data)

    rounded_predictions = torch.round(torch.sigmoid(test_predictions))
    accuracy = accuracy_score(test_labels, rounded_predictions)
    return accuracy

accuracy = test_accuracy(net, valid_data_2d, valid_label)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 81.28%


### Using Convultional Layers